# Import Module

In [ ]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, Dense, GlobalAveragePooling2D
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Input Dataset

## Getting Dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

## Analyzing Dataset (Original)

In [ ]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(train_images[i])
    print(train_labels[i], end=", ")
plt.show()

## Preprocessing Dataset

In [ ]:
train_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, width_shift_range=0.125, height_shift_range=0.125, horizontal_flip=True)
test_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
for data in (train_gen, test_gen):
    data.fit(train_images)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

## Analyzing Dataset (Normalized)

In [ ]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(train_images[i])
    print(train_labels[i])
plt.show()

# Model

## Generating Model

In [ ]:
# Convolutional Layer
def conv(filters, kernel_size, strides=1):
    return Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))

# Residual Block
def residual_block_a(filters, strides):
    def f(x):
        x = BatchNormalization()(x)
        b = Activation('relu')(x)

        x = conv(filters // 4 , 1, strides)(b)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = conv(filters // 4, 3)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = conv(filters, 1)(x)

        sc = conv(filters, 1, strides)(b)

        return Add()([x, sc])
    return f

def residual_block_b(filters):
    def f(x):
        sc = x

        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = conv(filters // 4, 1)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = conv(filters // 4, 3)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = conv(filters, 1)(x)

        return Add()([x, sc])
    return f

def residual_block(filters, strides, unit_size):
    def f(x):
        x = residual_block_a(filters, strides)(x)
        for i in range(unit_size - 1):
            x = residual_block_b(filters)(x)
        return x
    return f

# Model
## Input
input = Input(shape=(32, 32, 3))

## Extracting Feature
x = conv(16, 3)(input)
x = residual_block(64, 1, 18)(x)
x = residual_block(128, 2, 18)(x)
x = residual_block(256, 2, 18)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)

## Classifying
output = Dense(10, activation='softmax', kernel_regularizer=l2(0.0001))(x)

model = Model(inputs=input, outputs=output)

## Compiling Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(momentum=0.9), metrics=['acc'])

# Training

In [ ]:
def step_scheduler(epoch):
    x = 0.1
    if epoch >= 80: x = 0.01
    if epoch >= 120: x = 0.001
    return x

lr_scheduler = LearningRateScheduler(step_scheduler)

history = model.fit_generator(train_gen.flow(train_images, train_labels, batch_size=128), epochs=150, steps_per_epoch=train_images.shape[0] // 128, validation_data=test_gen.flow(test_images, test_labels, batch_size=128), validation_steps=test_images.shape[0] // 128, callbacks=[lr_scheduler])

In [ ]:
from keras.models import load_model

model.save('ResNet_CIFAR10.h5')
model = load_model('ResNet_CIFAR10.h5')

## Analyzing Training Result

In [ ]:
plt.plot(history.history['acc'], label='acc')
plt.plot(history.history['val_acc'], label='val_acc')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()

# Evaluation

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_gen.flow(test_images, test_labels, batch_size=128), steps=10)
print('loss: {:.4f}\nacc: {:.4f}'.format(test_loss, test_acc))

# Prediction

In [ ]:
test_predictions = model.predict_generator(test_gen.flow(test_images[:5], shuffle=False, batch_size=1), steps=10)
test_predictions = np.argmax(test_predictions, axis=1)
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

for i in range(5):
    plt.subplot(2, 5, i + 1)
    plt.imshow(test_images[i])
    print([labels[test_predictions[i]]], end=", ")
plt.show()